In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

from util import imshow, load_img, save_img, apply_lum, match_lum
from model import StyleTransferModel
from losses import style_loss
from train import train

# https://images1.novica.net/pictures/10/p348189_2a.jpg, https://www.novica.com/p/impressionist-painting-in-delod-pangkung/348189/
style_path = tf.keras.utils.get_file('impressionist-bali.jpg','file:///home/jupyter/pictures/impressionist-bali.jpg')

style_img = load_img(style_path, max_dim=512)
imshow(style_img, 'Style Image')

plt.show()

In [ ]:
# reconstruct style, for every layer
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']


for idx in range(len(style_layers)):
    extractor = StyleTransferModel(style_layers[:idx+1], ['block1_conv1'])

    style_targets = extractor(style_img)['style']
    style_weights = [ 1.0, 1.0, 1.0, 1.0, 1.0 ]
    style_weights = style_weights[:idx+1]
    style_weights = [ w/sum(style_weights) for w in style_weights ] # normalize weights

    # initialize the gradients with random noise
    initial_gradients = tf.Variable(tf.random.uniform(style_img.shape))

    def loss_func(image):
        outputs = extractor(image)
        loss = style_loss(outputs['style'], style_targets, style_weights)
        return loss

    result = train(loss_func, extractor, initial_gradients, epochs=20)

    save_img(result[0], 'style_{}.png'.format(style_layers[idx]))